In [1]:
import pandas as pd
#set root directory to parent from this current directory
import os
os.chdir(os.path.dirname(os.getcwd()))
import requests


In [2]:
with open(r"C:\Users\cruz" + r'\API_openAI.txt', 'r') as f:
    read_api_key = f.readline()

In [53]:
df_template = pd.read_pickle(r"data\output\embeddings_mpnet_temp.pkl")

In [54]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

import plotly.express as px
import plotly.graph_objects as go

# Assuming df['embedding'] is a list of embeddings (e.g., 768-dimensional vectors)
# Convert the embeddings to a numpy array
embeddings = np.vstack(df_template['embedding'].values)

# Perform dimensionality reduction with t-SNE to reduce to 2D
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

# Add the 2D embeddings back to the dataframe for visualization
df_template['embedding_2d_x'] = embeddings_2d[:, 0]
df_template['embedding_2d_y'] = embeddings_2d[:, 1]



KeyError: 'embedding'

In [51]:
# Plot using Plotly Express
fig = px.scatter(df_template, x='embedding_2d_x', y='embedding_2d_y',
                 title="2D Visualization of Embeddings",
                    hover_data={"combined"},
                 labels={'embedding_2d_x': 'Dimension 1', 'embedding_2d_y': 'Dimension 2'},
                 color='Famille')

# Calculate the centroid (mean position) of each "Famille" in the 2D space
centroids = df_template.groupby('Famille')[['embedding_2d_x', 'embedding_2d_y']].mean().reset_index()

# Calculate the size of the label based on the number of points in each "Famille"
famille_sizes = df_template['Famille'].value_counts().reset_index()
famille_sizes.columns = ['Famille', 'size']

# Merge the centroids with the sizes to adjust the label size
centroids = centroids.merge(famille_sizes, on='Famille')

# Add text labels for each "Famille" at its centroid position
for i, row in centroids.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['embedding_2d_x']],
        y=[row['embedding_2d_y']],
        text=[row['Famille']],
        mode='text',
        textfont=dict(size=row['size'] * 2),  # Adjust size based on the number of points
        showlegend=False
    ))

# Show the plot
fig.show()

c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [14]:
# Import libraries
import requests
import numpy as np
import pandas as pd
import sys
from more_itertools import chunked
from sklearn.cluster import MiniBatchKMeans
import plotly.express as px
from sentence_transformers import SentenceTransformer
import pandas as pd


c:\Users\cruz\AppData\Local\anaconda3\envs\Carbon_AI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [26]:
df_original = pd.read_excel(r"data\input_data\achats_EPFL\Catalyse_Ligne articles 2023_TOTAL EPFL.xlsx")


In [27]:
df = df_original[df_original["Entité de gestion"] == "002000School of Life Sciences"].drop_duplicates().reset_index().sample(1000)

In [29]:
df.head()

,index,Code CDE,Date de commande,Référence article,Fournisseur,Libellé article,Unité Organisationnelle,Devise,PU commande,Qté commande,...,Familles Niveau 2,Statut,Type de DA,Entité de gestion,orga_com_order,orga_ent_order,orga_site_order,orga_order,orga_id,short_label
1419,5179,400369684,2023-01-17,10447080,219,1KG D(+)- Glucose Monohydrate BAKER ANALYZED R...,Unité du Prof. La Manno,CHF,49.23,1.0,...,3.0,En cours de commande,sciquest_catalog,002000School of Life Sciences,003000Ecole Polytechnique de Lausanne,004000Brain Mind Institute,005000Prof. La Manno Group,005000Prof. La Manno Group,1796,UPLAMANNO
30095,108582,400429499,2023-09-27,318111,836,PAQ25 CHEM C LYR PREM BLEU 11073780,Unité du Prof. Lutolf,CHF,3.60,1.0,...,21.0,En cours de commande,extern_catalog,002000School of Life Sciences,003000Ecole Polytechnique de Lausanne,004000Interschool Institute of Bioengineering ...,005000Prof. Lutolf Group,005000Prof. Lutolf Group,2394,UPLUT
37059,133703,400442725,2023-11-17,05277990001,219,CONFIRM ANTI-PR (1E2),Unité de la Prof. Brisken,CHF,550.00,1.0,...,19.0,En cours de commande,sciquest_catalog,002000School of Life Sciences,003000Ecole Polytechnique de Lausanne,004000Swiss Institute for Experimental Cancer ...,005000Prof. Brisken Group,005000Prof. Brisken Group,2370,UPBRI
16635,61650,400402818,2023-05-30,520185,262,microTUBE-500 AFA Fiber Screw- Cap (25),Unité du Prof. Radtke,EUR,269.00,4.0,...,NaN,En cours de commande,buy,002000School of Life Sciences,003000Ecole Polytechnique de Lausanne,004000Swiss Institute for Experimental Cancer ...,005000Prof. Radtke Group,005000Prof. Radtke Group,2402,UPRAD
1218,4369,400369223,2023-01-15,211-2129,219,"Tube Safelock 2.0ml Rainbow Pack, UoM: 1 1.000...",Unité de la Prof. Gisou van der Goot,CHF,69.16,1.0,...,18.0,En cours de commande,sciquest_catalog,002000School of Life Sciences,003000Ecole Polytechnique de Lausanne,004000Global Health Institute,005000Prof. van der Goot Group,005000Prof. van der Goot Group,2420,VDG


In [30]:
df["text"] = df["Libellé article"] + " " + df["Famille"]

In [31]:
df["text"]

1419     1KG D(+)- Glucose Monohydrate BAKER ANALYZED R...
30095    PAQ25 CHEM C LYR PREM BLEU 11073780 Accessoire...
37059    CONFIRM ANTI-PR (1E2) Produits de suture médic...
16635    microTUBE-500 AFA Fiber Screw- Cap (25) Équipe...
1218     Tube Safelock 2.0ml Rainbow Pack, UoM: 1 1.000...
                               ...                        
27835    05_fragCry1Venus_FOR;DNA-synthesis 38625 Fourn...
23916    SC1626 Mutagenesis:pET29b-TotA R76D K77E Targe...
33735                         Triton(TM) X-100 Surfacteurs
10398    code775 Loc: C4-4 Filtre Stericup-GP 0.22micro...
6496     Adhesive aluminium PCR Foil Seal (135mm x 80mm...
Name: text, Length: 1000, dtype: object

In [32]:
batch_size = 64

model = SentenceTransformer('all-mpnet-base-v2', device='cpu')

embeddings = []
for batch_text in chunked(df['text'].tolist(), batch_size):
    embeddings.extend(model.encode(batch_text, show_progress_bar=False))
    sys.stdout.write(f'\rprocessed for {len(embeddings)} works so far... (out of {df.shape[0]})')
    sys.stdout.flush()

embeddings = pd.DataFrame(embeddings, index=df.index)
embeddings.to_pickle('data/output/sample_SV_1000.pkl')

processed for 1000 works so far... (out of 1000)

In [46]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
1419,0.004837,-0.043324,0.030722,0.005353,-0.011821,-0.003475,-0.045990,0.027302,0.100840,-0.017800,...,-0.016984,-0.011189,0.046705,0.015515,0.022401,-0.033403,-0.006658,0.032575,-0.030485,0.024166
30095,0.057422,-0.064960,0.009479,0.071891,-0.049539,0.010212,-0.006256,0.044116,0.035753,-0.007000,...,-0.029097,-0.020320,-0.053384,-0.017962,-0.040405,0.039769,0.004800,0.015020,-0.090999,-0.031756
37059,-0.013315,-0.077190,0.019967,-0.011411,-0.046317,-0.025712,0.030896,-0.009542,-0.010570,-0.032571,...,-0.082060,-0.039444,-0.050130,0.013852,-0.003346,0.072097,-0.046098,-0.001136,-0.027623,-0.007114
16635,-0.040176,-0.095186,-0.005494,0.037761,0.015166,-0.029424,0.071834,-0.039032,0.031631,-0.018981,...,-0.043439,0.000126,0.025105,-0.028467,0.022703,-0.041000,0.033013,0.032413,-0.060949,-0.021654
1218,0.036742,-0.100802,-0.018065,0.017228,-0.016738,0.008368,0.064605,-0.009954,-0.011448,0.012513,...,-0.034413,0.023326,0.020342,0.001134,0.016093,-0.044275,-0.003831,-0.001796,-0.046466,-0.007435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27835,0.033574,-0.014208,0.000902,-0.009746,-0.051076,0.032646,0.034221,0.003513,-0.018104,-0.070112,...,-0.042790,-0.031809,0.005864,-0.017028,-0.001266,0.002702,0.000669,0.032523,-0.021458,-0.025745
23916,-0.000887,-0.059469,0.035540,0.015135,0.012598,0.013568,-0.014302,0.026083,0.016095,-0.000315,...,-0.066303,0.040851,-0.070686,0.039556,-0.015277,0.006357,0.026885,-0.040896,-0.025487,0.022504
33735,0.040125,-0.112656,0.009105,0.000120,-0.016832,-0.023168,0.026873,0.014081,0.010421,-0.021760,...,-0.075677,-0.021877,0.056161,0.022168,0.028974,-0.022717,-0.043487,-0.017266,-0.016742,0.038670
10398,0.019068,-0.065062,-0.002974,0.029213,-0.046251,-0.005345,0.034036,0.029401,0.061111,0.013081,...,-0.044436,0.006931,0.022666,0.006372,-0.019497,-0.001017,0.048937,0.012601,-0.084777,-0.014920


In [33]:
n_clusters = 12
kmeans = MiniBatchKMeans(n_clusters=n_clusters, n_init='auto')
batch_size = 10000
for i in range(0, len(embeddings), batch_size):
    batch = embeddings.iloc[i:i+batch_size]
    kmeans.partial_fit(batch)

df['cluster'] = kmeans.predict(embeddings)


In [34]:
n_representatives = 10
representatives = {}

for cluster in df['cluster'].unique():
    cluster_embeddings = embeddings[df['cluster'] == cluster]

    # If there are fewer samples than n_representatives, use all samples
    if len(cluster_embeddings) < n_representatives:
        global_indices = cluster_embeddings.index
    else:
        # Use Kmeans to select n_representatives
        subcluster = MiniBatchKMeans(n_clusters=n_representatives, n_init='auto')
        subcluster.fit(cluster_embeddings)

        # Get distance of each point to each cluster
        distances = subcluster.transform(cluster_embeddings)

        # Retrieve indices of samples closest to centroids
        closest_indices = np.argmin(distances, axis=0)
        global_indices = cluster_embeddings.iloc[closest_indices].index

    # Align with global indices and collect the representative texts
    representatives[int(cluster)] = df.loc[global_indices, 'text'].str.slice(0, 400).tolist()

representatives


{9: ['DMEM, high glucose, no glutamine Fournitures pour culture de tissus et criblage à haut débit',
  'Water Produits biochimiques',
  'DMEM/F-12, GlutaMAX Supplement-500 mL Loc: F1-2 4deg Produits biochimiques',
  '16% FORMALDEHYDE (W/V), METHANOL-FREE 10 X 1 ML, UoM: 1 1 SET, CAS: 50-00-0, 1 1 SET Produits biochimiques',
  'Glycerol Dérivés organiques et composés avec substitution',
  'L-Glutamic acid monopotassium salt monohydrate Produits biochimiques',
  'DMEM, high glucose, pyruvate-500 mL Loc: F2-1 4deg Produits biochimiques',
  'DMEM high glucose GlutaMAX supplement- Fournitures pour culture de tissus et criblage à haut débit',
  'BMS-986235 Produits biochimiques',
  'Mettler-Toledo certified buffer pH 4.01, volume 6 x 250 mL Produits biochimiques'],
 3: ['code121 Loc: J4-3 Savon main Bactolin Pure 500ml 981389 Fournitures et matériels de laboratoire',
  'code738 Loc: M4-3 TipOne filter tips 1-200 ul Fournitures et matériels de laboratoire',
  'Code369   Gant Vasco Nitrile Ble

In [36]:
prompt

'I have 12 clusters of different purchases of a technical university. For each clusters, I give you a description and a categorization that are part of the cluster.\nFor each cluster, find a short title (3-5 words), and a description (2-3 sentences) that best describe the content of the clusters. Here are the clusters: \n\n\nCLUSTER: 9\n - DMEM, high glucose, no glutamine Fournitures pour culture de tissus et criblage à haut débit\n - Water Produits biochimiques\n - DMEM/F-12, GlutaMAX Supplement-500 mL Loc: F1-2 4deg Produits biochimiques\n - 16% FORMALDEHYDE (W/V), METHANOL-FREE 10 X 1 ML, UoM: 1 1 SET, CAS: 50-00-0, 1 1 SET Produits biochimiques\n - Glycerol Dérivés organiques et composés avec substitution\n - L-Glutamic acid monopotassium salt monohydrate Produits biochimiques\n - DMEM, high glucose, pyruvate-500 mL Loc: F2-1 4deg Produits biochimiques\n - DMEM high glucose GlutaMAX supplement- Fournitures pour culture de tissus et criblage à haut débit\n - BMS-986235 Produits bioc

[{'id':9, 'title':'Biochemical Products & Supplies', 'description':'This cluster includes various biochemical products and supplies necessary for tissue culture and screening. It features media, supplements, and buffers required for laboratory experiments.'}, {'id':3, 'title':'Laboratory Equipment & Supplies', 'description':'This cluster consists of essential laboratory supplies and equipment including gloves, pipette tips, and disinfectants. It is tailored for laboratory use ensuring safety and efficiency in various experimental procedures.'}, {'id':6, 'title':'Tissue Culture Essentials', 'description':'Focusing on tissue culture, this cluster comprises items such as centrifuge tubes, multiwell plates, and antibiotics. These products are vital for conducting biological assays and culture procedures.'}, {'id':11, 'title':'Advanced Laboratory Instruments', 'description':'This cluster encapsulates advanced laboratory instruments and consumables, including refrigeration units and filtrati

,cluster_id,cluster_title,cluster_description
0,9,Biochemical Products & Supplies,This cluster includes various biochemical prod...
1,3,Laboratory Equipment & Supplies,This cluster consists of essential laboratory ...
2,6,Tissue Culture Essentials,"Focusing on tissue culture, this cluster compr..."
3,11,Advanced Laboratory Instruments,This cluster encapsulates advanced laboratory ...
4,1,Nucleic Acid Processing Supplies,This cluster is dedicated to supplies for nucl...
5,4,DNA Synthesis Reagents,This cluster includes reagents and materials s...
6,5,DNA Sequencing Products,This cluster is focused on products related to...
7,8,Antibodies & Reagents,This cluster consists of antibodies and bioche...
8,10,Office & Laboratory Accessories,This cluster features various office supplies ...
9,2,Culture and Laboratory Materials,This cluster entails various materials and con...


In [40]:
# Join the description of the clusters obtained with ChatGPT to the work downloaded from OpenAlex
df = df.set_index('cluster').join(cluster_description.set_index('cluster_id')).reset_index()

In [41]:

global_distribution = df['cluster_title'].value_counts(normalize=True)

In [45]:
df['cluster_title']

0       Biochemical Products & Supplies
1       Laboratory Equipment & Supplies
2             Tissue Culture Essentials
3       Advanced Laboratory Instruments
4       Laboratory Equipment & Supplies
                     ...               
995              DNA Synthesis Reagents
996    Nucleic Acid Processing Supplies
997     Advanced Laboratory Instruments
998           Tissue Culture Essentials
999     Laboratory Equipment & Supplies
Name: cluster_title, Length: 1000, dtype: object

In [42]:
# Gray treemap showing the size of the subtopics from 'Natural Language Processing' discovered using Kmeans on the embeddings
def insert_line_breaks(text, n):
    words = text.split()
    lines = [' '.join(words[i:i+n]) for i in range(0, len(words), n)]
    return '<br>'.join(lines)

data = global_distribution.to_frame('global_frac')
data['color'] = 'gray'
data['title'] = data.index
data['label'] = data['title'].apply(lambda x: insert_line_breaks(x, 3))
data['label'] += '<br> ('+(data['global_frac']*100).round(1).astype(str)+'%)'

# Creating the treemap
fig = px.treemap(data, path=['label'], values=data['global_frac'],
                 title='Clusters in Natural Language Processing',
                 color='color', color_discrete_map={'gray': '#ccc'})

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25), width=1200, height=600)

# Display the treemap
fig.show()